<a href="https://colab.research.google.com/github/ML-Challenge/week5-preprocessing-and-tunning/blob/master/L2.Hyperparameter%20Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" /></a>

Building powerful machine learning models depends heavily on the set of hyperparameters used. But with increasingly complex models with lots of options, how do we efficiently find the best settings for our particular problem? In this lesson we will get practical experience in using some common methodologies for automated hyperparameter tuning in Python using Scikit Learn. These include Grid Search, Random Search & advanced optimization methodologies including Bayesian & Genetic algorithms. We will use a dataset predicting credit card defaults as we build skills to dramatically increase the efficiency and effectiveness of our machine learning model building.

# Setup

In [1]:
# Download lesson datasets
# Required if you're using Google Colab
#!wget "https://github.com/ML-Challenge/week5-preprocessing-and-tunning/raw/master/datasets.zip"
#!unzip -o datasets.zip

In [ ]:
# Import utils
# We'll be using this module throughout the lesson
import utils

In [ ]:
# Import dependencies
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# and setting the size of all plots.
plt.rcParams['figure.figsize'] = [11, 7]

# Hyperparameters and Parameters

In this introductory chapter we will learn the difference between hyperparameters and parameters. We will practice extracting and analyzing parameters, setting hyperparameter values for several popular machine learning algorithms. Along the way we will learn some best practice tips & tricks for choosing which hyperparameters to tune and what values to set & build learning curves to analyze your hyperparameter choices.

## Introduction & 'Parameters'

## Parameters in Logistic Regression

### Extracting a Logistic Regression parameter

### Extracting a Random Forest parameter

## Introducing Hyperparameters

### Hyperparameters in Random Forests

### Exploring Random Forest Hyperparameters

### Hyperparameters of KNN

## Setting & Analyzing Hyperparameter Values

### Automating Hyperparameter Choice

### Building Learning Curves

# Grid search

This chapter introduces a popular automated hyperparameter tuning methodology called Grid Search. We will learn what it is, how it works and practice undertaking a Grid Search using Scikit Learn. We will then learn how to analyze the output of a Grid Search & gain practical experience doing this.

## Introducing Grid Search

### Build Grid Search functions

### Iteratively tune multiple hyperparameters

### How Many Models?

## Grid Search with Scikit Learn

### GridSearchCV inputs

### GridSearchCV with Scikit Learn

## Understanding a grid search output

### Using the best outputs

### Exploring the grid search results

### Analyzing the best results

### Using the best results

# Random Search

This chapter introduces another popular automated hyperparameter tuning methodology called Random Search. We will learn what it is, how it works and importantly how it differs from grid search. We will learn some advantages and disadvantages of this method and when to choose this method compared to Grid Search. We will practice undertaking a Random Search with Scikit Learn as well as visualizing & interpreting the output.

## Introducing Random Search

### Randomly Sample Hyperparameters

### Randomly Search with Random Forest

### Visualizing a Random Search

## Random Search in Scikit Learn

### RandomSearchCV inputs

### The RandomizedSeachCV Class

### RandomSearchCV in Scikit Learn

## Comparing Grid and Random Search

### Comparing Random & Grid Search

### Grid and Random Search Side by Side

# Informed Search

This final chapter gives a taste of more advanced hyperparameter tuning methodologies known as `informed search`. This includes a methodology known as Coarse To Fine as well as Bayesian & Genetic hyperparameter tuning algorithms. We will learn how informed search differs from uninformed search and gain practical skills with each of the mentioned methodologies, comparing and contrasting them as you go.

## Informed Search: Coarse to Fine

### Visualizing Coarse to FIne

### Coarse to Fine Iterations

## Informed Search: Bayesian Statistics

### Bayes Rule in Python

### Bayesian Hyperparameter tuning with Hyperopt

## Informed Search: Genetic Algorithms

### Genetic Hyperparameter Tuning with TPOT

### Analyzing TPOT's stability

## Congratulations!